In [35]:
import pandas as pd

portfolio = pd.read_excel("data/portfolio.xlsx", sheet_name="Stocks")

Let's update Company Name, Market Price and Annual Dividend per Share
Notes:
- Market Price can be extracted from <ticker>.info["currentPrice"] for regular stocks, but not for ETFs
- For ETF there is <ticker>.info["navPrice"], which may slightly differ comparing to market price
- ETF net asset is a value of all assets of the fund, substracting liabilities. For ETFs NAV price is calculated every day
- Stocks dividends can be returned using dividendRate, but not for ETF. Ideally for ETFs we should calculate TTM dividend rate (Trailing twelwe months)
- yfinance does have <ticker>.info["trailingAnnualDividendRate"] but it's 0 for SCHD. Not reliable

In [68]:
import yfinance as yf
from main import calculate_etf_ttm

for i in range(len(portfolio)):
  ticker_data = yf.Ticker(portfolio.loc[i, "Ticker"])
  match ticker_data.info["quoteType"]:
    case "ETF":
      annual_dividend_per_share = calculate_etf_ttm(ticker_data)
      current_price = ticker_data.info["navPrice"]
    case _:
      annual_dividend_per_share = ticker_data.info["dividendRate"]
      current_price = ticker_data.info["currentPrice"]

  portfolio.loc[i, "Company Name"] = ticker_data.info["shortName"]
  portfolio.loc[i, "Market Price"] = round(current_price,2)
  portfolio.loc[i, "Annual Dividend per Share"] = annual_dividend_per_share



In [69]:
portfolio.head()

,Company Name,Ticker,Number of Shares,Buy Price,Market Price,Annual Dividend per Share,Date of Purchase,Capital Gains,Dividends Paid,Total Return
0,Atlantica Sustainable Infrastru,AY,687,23.55,21.60,1.78,22.06.2023,NaN,NaN,NaN
1,Atlantica Sustainable Infrastru,AY,400,24.66,21.60,1.78,27.07.2023,NaN,NaN,NaN
2,Schwab US Dividend Equity ETF,SCHD,100,24.66,75.82,2.66,27.07.2024,NaN,NaN,NaN


In [55]:
portfolio.to_excel("data/portfolio.xlsx", sheet_name="Stocks", index=False)